# K-Means analysis 

Los datos para este trabajo fueron tomados de la plataforma Kaggle:
https://www.kaggle.com/datasets/nanditapore/credit-risk-analysis?resource=download

## ¿Qué es K-means?
**K-means** es un algoritmo de clustering (agrupamiento) que se utiliza en aprendizaje no supervisado para dividir un conjunto de datos en grupos o clústeres basados en la similitud de sus elementos. El objetivo del algoritmo K-means es particionar los datos en K clústeres de tal manera que cada punto de datos pertenezca al clúster cuyo centroide (punto central) es el más cercano. En resumen, K-means busca encontrar K centroides que minimicen la suma de las distancias cuadradas entre los puntos de datos y sus centroides asignados.

**K-means++**, por otro lado, es una mejora del algoritmo K-means que se utiliza para inicializar los centroides de manera más eficiente y efectiva. En K-means tradicional, la inicialización de los centroides puede ser aleatoria, lo que puede llevar a soluciones subóptimas o convergencia a mínimos locales. K-means++ aborda este problema seleccionando los centroides iniciales de una manera que aumenta las posibilidades de converger a una solución de alta calidad.